In [1]:
import numpy as np
import scipy.stats as stats
import csv

In [2]:
def TSVtoList(tsvFile):
    import csv
    #Create a container for the tsv
    tsvList = []
    #Open the .tsv file and append it to a python list
    with open(tsvFile,'r') as tsvIn:
        tsvIn = csv.reader(tsvIn, delimiter='\t')
        for row in tsvIn:
            tsvList.append(row)
    return tsvList

In [3]:
def ExcludeNonImageRows(tsvList):
    index = []
    currentIndex = 1
    mediaNameIndex = GetIndexForHeader("MediaName")
    for row in tsvList[1:]:
        if row[mediaNameIndex] == "":
            index.append(currentIndex)
        currentIndex += 1
    for row in reversed(index):
        del tsvList[row]
    return tsvList[1:]

In [4]:
def GetIndexForHeader(name):
    indexForHeader = dataHeaders.index(name)
    return indexForHeader

In [5]:
def DivideIntoSeparateImageLists(tsvList):
    imageDictionary = {}
    mediaNameIndex = GetIndexForHeader("MediaName")
    for row in tsvList[1:]:
        if row[113] not in imageDictionary:
            imageDictionary[row[mediaNameIndex]] = [row]
        else:
            imageDictionary[row[mediaNameIndex]].append(row)
    return imageDictionary

In [6]:
def CalculateDurationForImage(dataList):
    RecordingTimestampIndex = GetIndexForHeader("RecordingTimestamp")
    start =  float(dataList[0][RecordingTimestampIndex])
    end = float(dataList[-1][RecordingTimestampIndex])
    duration = end - start
    #Convert to seconds
    duration = duration
    return duration

In [7]:
def CalculateFixation(dataList, dataHeaders):
    fixations = {}
    fixationIndex = GetIndexForHeader("FixationIndex")
    RecordingTimestampIndex = GetIndexForHeader("RecordingTimestamp")
    fixations[0] = []
    fixations[0].append(dataHeaders)
    for row in dataList:
        if row[fixationIndex] not in fixations.keys():
            #print("RR",row[fixationIndex])
            fixations[row[fixationIndex]] = []
            fixations[row[fixationIndex]].append(row)
        elif row[fixationIndex] in fixations.keys():
            fixations[row[fixationIndex]].append(row)
    fixations.pop('', None)
    #print("FIXX: ", fixations.get(fixations.keys))
    fixationCount = len(fixations.keys())
    fixationDuration = []
    for key in fixations.keys():
        if key != 0:
            start = fixations[key][0][RecordingTimestampIndex]
            end = fixations[key][-1][RecordingTimestampIndex]
            duration = int(end) - int(start)
            fixationDuration.append(duration)
    fixationDuration = [x for x in fixationDuration if x != 0]
    fixationDurationAverage = np.mean(fixationDuration)
    fixationDurationTotal = np.sum(fixationDuration)
    return fixationCount, fixationDurationAverage, fixationDurationTotal, fixations

In [8]:
def writeToFile(dataList, fileName):
    fixationIndex = GetIndexForHeader("FixationIndex")
    with open(fileName, 'w') as outcsv:   
        writer = csv.writer(outcsv, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        for key in dataList.keys():
            writer.writerow(dataList[key][0])

In [9]:
def filterAOI(dataList, aoi):
    newDataList = []
    for i in range(len(dataList)):
        if "1" in dataList[i][aoi]:
            newDataList.append(dataList[i])
    return newDataList

In [10]:
def sumColumn(AllSum, columnIndex, fixations, screenIDs):
    #print(columnIndex)
    MediaNameIndex = GetIndexForHeader("MediaName")
    columnName = dataHeaders[MediaNameIndex]
    #dur = 0
    durIndex = GetIndexForHeader("GazeEventDuration")
    for key in fixations.keys():
        if key != 0:
            if fixations[key][0][columnIndex] is not "":
                if fixations[key][0][MediaNameIndex] not in AllSum.keys():
                    AllSum[fixations[key][0][MediaNameIndex]] = []
                    AllSum[fixations[key][0][MediaNameIndex]].append(fixations[key][0])
                    AllSum[fixations[key][0][MediaNameIndex]][0][columnIndex] = fixations[key][0][columnIndex]
                else:
                    if AllSum[fixations[key][0][MediaNameIndex]][0][columnIndex].strip() is not "":
                        AllSum[fixations[key][0][MediaNameIndex]][0][columnIndex] = str(int(fixations[key][0][columnIndex]) + int(AllSum[fixations[key][0][MediaNameIndex]][0][columnIndex]))
    return AllSum

In [11]:
def CalculateFixationDuration(dataList, columnName):
    index = 0
    FixationIndexIndex = GetIndexForHeader("FixationIndex")
    RecordingTimestampIndex = GetIndexForHeader("RecordingTimestamp")
    columnIndex = GetIndexForHeader(columnName)
    prev = "0"
    prevCol = 0
    current = "0"
    currentCol = 0
    start = 0
    end = 0
    fixationDurations = []
    for row in dataList:
        prev = current
        current = row[FixationIndexIndex]
        
        prevCol = currentCol
        currentCol = row[columnIndex]
        if row[columnIndex].strip() is not "":
            if prev is "" and current is not "":
                start = row[RecordingTimestampIndex]
            elif prev is not "" and current is "":
                end = row[RecordingTimestampIndex]
                duration = int(end) - int(start)
                duration = float(duration)
                fixationDurations.append(duration)
        else:
            print("NOT")
    return fixationDurations

In [12]:
def calDur(fixations, Durations, columnName):
    #print(columnName)
    MediaNameIndex = GetIndexForHeader("MediaName")
    #columnName = dataHeaders[MediaNameIndex]
    dur = 0
    durIndex = GetIndexForHeader("GazeEventDuration")
    columnIndex = GetIndexForHeader(columnName)
    for key in fixations.keys():
        if key != 0:
        #print(columnName, row[durIndex])
            if fixations[key][0][columnIndex] is not "":
                dur = dur + float(fixations[key][0][durIndex])
    Durations[columnName] = dur
    return Durations

In [13]:
def CalculatePupilDilation(dataList):
    pupilLeft = []
    pupilRight = []
    PupilLeftIndex = GetIndexForHeader("PupilLeft")
    PupilRightIndex = GetIndexForHeader("PupilRight")
    for row in dataList:
        try:
            pupilLeft.append(float(row[PupilLeftIndex].replace(',','.')))
            pupilRight.append(float(row[PupilRightIndex].replace(',','.')))
        except ValueError:
            continue
    return pupilLeft, pupilRight

In [15]:
def durPerAOI(AllSum, columnIndex, fixations, aoi):
    #print(columnIndex)
    MediaNameIndex = GetIndexForHeader("MediaName")
    columnName = dataHeaders[MediaNameIndex]
    #dur = 0
    durIndex = GetIndexForHeader("GazeEventDuration")
    for key in fixations.keys():
        if fixations[key][0][aoi] == "1":
            if key != 0:
                if fixations[key][0][columnIndex] is not "":
                    if fixations[key][0][MediaNameIndex] not in AllSum.keys():
                        AllSum[fixations[key][0][MediaNameIndex]] = []
                        AllSum[fixations[key][0][MediaNameIndex]].append(fixations[key][0])
                        AllSum[fixations[key][0][MediaNameIndex]][0][aoi] = fixations[key][0][columnIndex]
                    else:
                        if AllSum[fixations[key][0][MediaNameIndex]][0][aoi].strip() is not "":
                            AllSum[fixations[key][0][MediaNameIndex]][0][aoi] = str(int(fixations[key][0][columnIndex]) + int(AllSum[fixations[key][0][MediaNameIndex]][0][aoi]))
    return AllSum

In [14]:
import numpy as np
for s in range(1,9):
    filePath = "/Users/Adil/Documents/MSc_Thesis/Script/" 
    inputFileName = "Input/PupilTrans/P1S ("+str(s)+").tsv" 
    dataList = TSVtoList(filePath + inputFileName)
    dataHeaders = dataList[0]
    dataList = ExcludeNonImageRows(dataList)
    pL, pR = CalculatePupilDilation(dataList)
    print(np.mean(pR))
print("DONE")

3.19980474629
3.1048087053
3.08166258295
3.09923446667
3.1296101949
3.01467606362
3.09711181577
2.99306044078
DONE


In [425]:
filePath = "/Users/Adil/Documents/MSc_Thesis/Script/" 
for p in range(1,13):
    if p not in  [3, 6, 8]:
        count = 5 
        print("Participant: ", p)
        for i in range(1,9): 
            inputFileName = "Input/Fixations/P" + str(p) + "/P" + str(p) + " (" + str(i) + ").tsv" 
            dataList = TSVtoList(filePath + inputFileName) 
            dataHeaders = dataList[0] 
            dataList = ExcludeNonImageRows(dataList)
            a,b,c, fixations = CalculateFixation(dataList, dataHeaders)
            if i == 1:
                AOIs = ['AOI[Screentitle]Hit', 'AOI[selection]Hit', 'AOI[title]Hit', 'AOI[text]Hit', 'AOI[arrowleft]Hit', 'AOI[pagecount]Hit', 'AOI[rating]Hit', 'AOI[likes]Hit', 'AOI[comments]Hit', 'AOI[idea]Hit', 'AOI[fullscreen]Hit', 'AOI[arrowright]Hit', 'AOI[date]Hit', 'AOI[date 2]Hit', 'AOI[likes 2]Hit', 'AOI[arrowleft 2]Hit', 'AOI[arrowright 2]Hit', 'AOI[selection 2]Hit', 'AOI[comments 2]Hit', 'AOI[pagecount 2]Hit', 'AOI[rating 2]Hit', 'AOI[text 2]Hit', 'AOI[title 2]Hit', 'AOI[idea 2]Hit', 'AOI[date 3]Hit', 'AOI[likes 3]Hit', 'AOI[arrowleft 3]Hit', 'AOI[arrowright 3]Hit', 'AOI[selection 3]Hit', 'AOI[comments 3]Hit', 'AOI[pagecount 3]Hit', 'AOI[rating 3]Hit', 'AOI[text 3]Hit', 'AOI[title 3]Hit', 'AOI[idea 3]Hit', 'AOI[date 4]Hit', 'AOI[likes 4]Hit', 'AOI[arrowleft 4]Hit', 'AOI[arrowright 4]Hit', 'AOI[selection 4]Hit', 'AOI[comments 4]Hit', 'AOI[pagecount 4]Hit', 'AOI[rating 4]Hit', 'AOI[text 4]Hit', 'AOI[title 4]Hit', 'AOI[idea 4]Hit', 'AOI[reviews]Hit', 'AOI[reviews 2]Hit', 'AOI[reviews 3]Hit', 'AOI[reviews 4]Hit']    
            else:
                AOIs = [s for s in dataHeaders if "AOI" in s and "Hit" in s and (str(count) == s[len(s)-6: len(s)-4].strip() or str(count+1)== s[len(s)-6: len(s)-4].strip() or str(count+2) == s[len(s)-6: len(s)-4].strip()  or str(count+3) == s[len(s)-6: len(s)-4].strip())]
                AOIs.append('AOI[fullscreen ' + str(i) + ']Hit')
                count = count + 4
            AllSum = {}
            Durations = {}
            AllSum[0] = fixations[0]
            
            for aoi in AOIs:
                AllSum = durPerAOI(AllSum, gazeIndex, fixations, GetIndexForHeader(aoi))
            writeToFile(AllSum, filePath+"Output/P" + str(p) +"_Fix_Durrrrrr"+str(i)+".csv")

inputFileName = "Input/EXPp2s3.tsv" 
dataList = TSVtoList(filePath + inputFileName) 
dataHeaders = dataList[0] 
dataList = ExcludeNonImageRows(dataList)
a,b,c, fixations = CalculateFixation(dataList, dataHeaders)
print("DONE")

Participant:  1


/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Participant:  2
Participant:  4
Participant:  5
Participant:  7
Participant:  9
Participant:  10
Participant:  11
Participant:  12
idea ; 0.12 0.12 0.25 0.05
fullscreen ; 0.14 0.14 1.96 0.04
idea 2 ; 0.17 0.17 1.96 0.04
idea 3 ; 0.13 0.13 0.39 0.04
idea 4 ; 0.15 0.15 1.02 0.04
DONE


In [410]:
filePath = "/Users/Adil/Documents/MSc_Thesis/Script/"
inputFileName = "Input/Fixations/P1/P1 (1).tsv"

dataList = TSVtoList(filePath + inputFileName)

dataHeaders = dataList[0]

dataList = ExcludeNonImageRows(dataList)

mediaNameIndex = GetIndexForHeader("MediaName")
gazeIndex = GetIndexForHeader("GazeEventDuration")
screenIDs = set()
a,b,c, fixations = CalculateFixation(dataList, dataHeaders)
AOIs = ['AOI[Screentitle]Hit', 'AOI[selection]Hit', 'AOI[title]Hit', 'AOI[text]Hit', 'AOI[arrowleft]Hit', 'AOI[pagecount]Hit', 'AOI[rating]Hit', 'AOI[likes]Hit', 'AOI[comments]Hit', 'AOI[idea]Hit', 'AOI[fullscreen]Hit', 'AOI[arrowright]Hit', 'AOI[date]Hit', 'AOI[date 2]Hit', 'AOI[likes 2]Hit', 'AOI[arrowleft 2]Hit', 'AOI[arrowright 2]Hit', 'AOI[selection 2]Hit', 'AOI[comments 2]Hit', 'AOI[pagecount 2]Hit', 'AOI[rating 2]Hit', 'AOI[text 2]Hit', 'AOI[title 2]Hit', 'AOI[idea 2]Hit', 'AOI[date 3]Hit', 'AOI[likes 3]Hit', 'AOI[arrowleft 3]Hit', 'AOI[arrowright 3]Hit', 'AOI[selection 3]Hit', 'AOI[comments 3]Hit', 'AOI[pagecount 3]Hit', 'AOI[rating 3]Hit', 'AOI[text 3]Hit', 'AOI[title 3]Hit', 'AOI[idea 3]Hit', 'AOI[date 4]Hit', 'AOI[likes 4]Hit', 'AOI[arrowleft 4]Hit', 'AOI[arrowright 4]Hit', 'AOI[selection 4]Hit', 'AOI[comments 4]Hit', 'AOI[pagecount 4]Hit', 'AOI[rating 4]Hit', 'AOI[text 4]Hit', 'AOI[title 4]Hit', 'AOI[idea 4]Hit', 'AOI[reviews]Hit', 'AOI[reviews 2]Hit', 'AOI[reviews 3]Hit', 'AOI[reviews 4]Hit']
AllSum = {}
AllSum[0] = fixations[0]
AllSum = durPerAOI(AllSum, gazeIndex, fixations, GetIndexForHeader('AOI[Screentitle]Hit'))
writeToFile(AllSum, filePath+"Output/GAZE_ST.csv")    
print("DONE")

314
DONE


/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [423]:
import pandas as pd

filePath = "/Users/Adil/Documents/MSc_Thesis/Script/" 
for p in range(1,13):
    if p not in  [3, 6, 8]:
        count = 5 
        print("Participant: ", p)
        for i in range(1,9): 
            print("Screen: ", i) 
            inputFileName = "Input/Fixations/P" + str(p) + "/P" + str(p) + " (" + str(i) + ").tsv" 
            dataList = TSVtoList(filePath + inputFileName) 
            dataHeaders = dataList[0] 
            dataList = ExcludeNonImageRows(dataList)
            #mediaNameIndex = GetIndexForHeader("MediaName")
            a,b,c, fixations = CalculateFixation(dataList, dataHeaders)
            if i == 1:
                AOIs = ['AOI[Screentitle]Hit', 'AOI[selection]Hit', 'AOI[title]Hit', 'AOI[text]Hit', 'AOI[arrowleft]Hit', 'AOI[pagecount]Hit', 'AOI[rating]Hit', 'AOI[likes]Hit', 'AOI[comments]Hit', 'AOI[idea]Hit', 'AOI[fullscreen]Hit', 'AOI[arrowright]Hit', 'AOI[date]Hit', 'AOI[date 2]Hit', 'AOI[likes 2]Hit', 'AOI[arrowleft 2]Hit', 'AOI[arrowright 2]Hit', 'AOI[selection 2]Hit', 'AOI[comments 2]Hit', 'AOI[pagecount 2]Hit', 'AOI[rating 2]Hit', 'AOI[text 2]Hit', 'AOI[title 2]Hit', 'AOI[idea 2]Hit', 'AOI[date 3]Hit', 'AOI[likes 3]Hit', 'AOI[arrowleft 3]Hit', 'AOI[arrowright 3]Hit', 'AOI[selection 3]Hit', 'AOI[comments 3]Hit', 'AOI[pagecount 3]Hit', 'AOI[rating 3]Hit', 'AOI[text 3]Hit', 'AOI[title 3]Hit', 'AOI[idea 3]Hit', 'AOI[date 4]Hit', 'AOI[likes 4]Hit', 'AOI[arrowleft 4]Hit', 'AOI[arrowright 4]Hit', 'AOI[selection 4]Hit', 'AOI[comments 4]Hit', 'AOI[pagecount 4]Hit', 'AOI[rating 4]Hit', 'AOI[text 4]Hit', 'AOI[title 4]Hit', 'AOI[idea 4]Hit', 'AOI[reviews]Hit', 'AOI[reviews 2]Hit', 'AOI[reviews 3]Hit', 'AOI[reviews 4]Hit']
                #for ao in AOIs:
                    #print(ao.strip().partition('[')[-1].rpartition(']')[0],";",end="",sep="")
            else:
                AOIs = [s for s in dataHeaders if "AOI" in s and "Hit" in s and (str(count) == s[len(s)-6: len(s)-4].strip() or str(count+1)== s[len(s)-6: len(s)-4].strip() or str(count+2) == s[len(s)-6: len(s)-4].strip()  or str(count+3) == s[len(s)-6: len(s)-4].strip())]
                AOIs.append('AOI[fullscreen ' + str(i) + ']Hit')
                count = count + 4
            AllSum = {}
            Durations = {}
            AllSum[0] = fixations[0]
            
            csv_file = "Output/P" + str(p) + "_Fix" + str(i) + ".csv" 
            df = pd.read_csv(filePath + csv_file)
            
            for aoi in AOIs:
                #AllSum = sumColumn(AllSum, GetIndexForHeader(aoi), fixations, screenIDs)
                colValue = df[aoi][0]
                print(df[aoi].name, end="", sep="")
                print(df[aoi][0])
                
                #print(colValue, end="", sep="")
            print()
print("DONE")

Participant:  1
Screen:  1
AOI[Screentitle]Hit3
AOI[selection]Hit2
AOI[title]Hit12
AOI[text]Hit106
AOI[arrowleft]Hit7
AOI[pagecount]Hit8
AOI[rating]Hit8
AOI[likes]Hit3
AOI[comments]Hit6
AOI[idea]Hit161
AOI[fullscreen]Hit314
AOI[arrowright]Hit2
AOI[date]Hit0
AOI[date 2]Hit0
AOI[likes 2]Hit1
AOI[arrowleft 2]Hit0
AOI[arrowright 2]Hit0
AOI[selection 2]Hit0
AOI[comments 2]Hit0
AOI[pagecount 2]Hit1
AOI[rating 2]Hit0
AOI[text 2]Hit17
AOI[title 2]Hit3
AOI[idea 2]Hit23
AOI[date 3]Hit0
AOI[likes 3]Hit0
AOI[arrowleft 3]Hit1
AOI[arrowright 3]Hit1
AOI[selection 3]Hit0
AOI[comments 3]Hit0
AOI[pagecount 3]Hit1
AOI[rating 3]Hit0
AOI[text 3]Hit56
AOI[title 3]Hit8
AOI[idea 3]Hit69
AOI[date 4]Hit0
AOI[likes 4]Hit0
AOI[arrowleft 4]Hit0
AOI[arrowright 4]Hit0
AOI[selection 4]Hit1
AOI[comments 4]Hit0
AOI[pagecount 4]Hit0
AOI[rating 4]Hit0
AOI[text 4]Hit48
AOI[title 4]Hit3
AOI[idea 4]Hit54
AOI[reviews]Hit0
AOI[reviews 2]Hit0
AOI[reviews 3]Hit0
AOI[reviews 4]Hit0

Screen:  2
AOI[selection 5]Hit1
AOI[title 5]Hi

/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1
AOI[comments 8]Hit0
AOI[pagecount 8]Hit1
AOI[rating 8]Hit0
AOI[text 8]Hit211
AOI[title 8]Hit17
AOI[idea 8]Hit233
AOI[reviews 5]Hit0
AOI[reviews 6]Hit0
AOI[reviews 7]Hit0
AOI[reviews 8]Hit0
AOI[Screentitle 5]Hitnan
AOI[fullscreen 5]Hitnan
AOI[Screentitle 6]Hitnan
AOI[fullscreen 6]Hitnan
AOI[Screentitle 7]Hitnan
AOI[fullscreen 7]Hitnan
AOI[Screentitle 8]Hitnan
AOI[fullscreen 8]Hitnan
AOI[fullscreen 2]Hit524

Screen:  3
AOI[selection 9]Hit0
AOI[title 9]Hit6
AOI[text 9]Hit68
AOI[arrowleft 9]Hit3
AOI[pagecount 9]Hit0
AOI[rating 9]Hit0
AOI[likes 9]Hit0
AOI[comments 9]Hit0
AOI[idea 9]Hit81
AOI[arrowright 9]Hit1
AOI[date 9]Hit0
AOI[date 10]Hit0
AOI[likes 10]Hit1
AOI[arrowleft 10]Hit0
AOI[arrowright 10]Hit0
AOI[selection 10]Hit0
AOI[comments 10]Hit0
AOI[pagecount 10]Hit0
AOI[rating 10]Hit1
AOI[text 10]Hit28
AOI[title 10]Hit6
AOI[idea 10]Hit41
AOI[date 11]Hit1
AOI[likes 11]Hit0
AOI[arrowleft 11]Hit0
AOI[arrowright 11]Hit0
AOI[selection 11]Hit1
AOI[comments 11]Hit0
AOI[pagecount 11]Hit0
AOI[rat

AOI[idea 19]Hit26
AOI[date 20]Hit0
AOI[likes 20]Hit0
AOI[arrowleft 20]Hit0
AOI[arrowright 20]Hit0
AOI[selection 20]Hit3
AOI[comments 20]Hit0
AOI[pagecount 20]Hit0
AOI[rating 20]Hit0
AOI[text 20]Hit23
AOI[title 20]Hit0
AOI[idea 20]Hit29
AOI[reviews 17]Hit0
AOI[reviews 18]Hit0
AOI[reviews 19]Hit0
AOI[reviews 20]Hit0
AOI[fullscreen 5]Hit263

Screen:  6
AOI[selection 21]Hit1
AOI[title 21]Hit14
AOI[text 21]Hit120
AOI[arrowleft 21]Hit0
AOI[pagecount 21]Hit0
AOI[rating 21]Hit0
AOI[likes 21]Hit0
AOI[comments 21]Hit0
AOI[idea 21]Hit137
AOI[arrowright 21]Hit0
AOI[date 21]Hit0
AOI[date 22]Hit0
AOI[likes 22]Hit0
AOI[arrowleft 22]Hit0
AOI[arrowright 22]Hit2
AOI[selection 22]Hit0
AOI[comments 22]Hit0
AOI[pagecount 22]Hit0
AOI[rating 22]Hit0
AOI[text 22]Hit137
AOI[title 22]Hit11
AOI[idea 22]Hit151
AOI[date 23]Hit1
AOI[likes 23]Hit0
AOI[arrowleft 23]Hit0
AOI[arrowright 23]Hit1
AOI[selection 23]Hit1
AOI[comments 23]Hit0
AOI[pagecount 23]Hit0
AOI[rating 23]Hit0
AOI[text 23]Hit96
AOI[title 23]Hit3
AOI[id

AOI[text 21]Hit47
AOI[arrowleft 21]Hit2
AOI[pagecount 21]Hit1
AOI[rating 21]Hit2
AOI[likes 21]Hit0
AOI[comments 21]Hit0
AOI[idea 21]Hit55
AOI[arrowright 21]Hit0
AOI[date 21]Hit0
AOI[date 22]Hit0
AOI[likes 22]Hit2
AOI[arrowleft 22]Hit7
AOI[arrowright 22]Hit0
AOI[selection 22]Hit0
AOI[comments 22]Hit2
AOI[pagecount 22]Hit4
AOI[rating 22]Hit0
AOI[text 22]Hit132
AOI[title 22]Hit2
AOI[idea 22]Hit171
AOI[date 23]Hit0
AOI[likes 23]Hit0
AOI[arrowleft 23]Hit3
AOI[arrowright 23]Hit0
AOI[selection 23]Hit0
AOI[comments 23]Hit0
AOI[pagecount 23]Hit0
AOI[rating 23]Hit0
AOI[text 23]Hit46
AOI[title 23]Hit9
AOI[idea 23]Hit66
AOI[date 24]Hit0
AOI[likes 24]Hit0
AOI[arrowleft 24]Hit0
AOI[arrowright 24]Hit0
AOI[selection 24]Hit5
AOI[comments 24]Hit0
AOI[pagecount 24]Hit0
AOI[rating 24]Hit0
AOI[text 24]Hit34
AOI[title 24]Hit21
AOI[idea 24]Hit76
AOI[reviews 21]Hit0
AOI[reviews 22]Hit0
AOI[reviews 23]Hit0
AOI[reviews 24]Hit0
AOI[fullscreen 6]Hit389

Screen:  7
AOI[selection 25]Hit0
AOI[title 25]Hit1
AOI[text 

AOI[arrowright 5]Hit0
AOI[date 5]Hit0
AOI[date 6]Hit0
AOI[likes 6]Hit3
AOI[arrowleft 6]Hit0
AOI[arrowright 6]Hit0
AOI[selection 6]Hit0
AOI[comments 6]Hit0
AOI[pagecount 6]Hit1
AOI[rating 6]Hit0
AOI[text 6]Hit80
AOI[title 6]Hit3
AOI[idea 6]Hit94
AOI[date 7]Hit0
AOI[likes 7]Hit0
AOI[arrowleft 7]Hit0
AOI[arrowright 7]Hit0
AOI[selection 7]Hit1
AOI[comments 7]Hit0
AOI[pagecount 7]Hit2
AOI[rating 7]Hit0
AOI[text 7]Hit171
AOI[title 7]Hit6
AOI[idea 7]Hit184
AOI[date 8]Hit0
AOI[likes 8]Hit1
AOI[arrowleft 8]Hit1
AOI[arrowright 8]Hit0
AOI[selection 8]Hit3
AOI[comments 8]Hit1
AOI[pagecount 8]Hit0
AOI[rating 8]Hit0
AOI[text 8]Hit337
AOI[title 8]Hit9
AOI[idea 8]Hit356
AOI[reviews 5]Hit0
AOI[reviews 6]Hit1
AOI[reviews 7]Hit0
AOI[reviews 8]Hit0
AOI[fullscreen 2]Hit928

Screen:  3
AOI[selection 9]Hit0
AOI[title 9]Hit19
AOI[text 9]Hit110
AOI[arrowleft 9]Hit6
AOI[pagecount 9]Hit0
AOI[rating 9]Hit1
AOI[likes 9]Hit0
AOI[comments 9]Hit0
AOI[idea 9]Hit145
AOI[arrowright 9]Hit0
AOI[date 9]Hit0
AOI[date 10]Hit

AOI[date 3]Hit0
AOI[likes 3]Hit1
AOI[arrowleft 3]Hit0
AOI[arrowright 3]Hit0
AOI[selection 3]Hit2
AOI[comments 3]Hit0
AOI[pagecount 3]Hit1
AOI[rating 3]Hit0
AOI[text 3]Hit73
AOI[title 3]Hit21
AOI[idea 3]Hit106
AOI[date 4]Hit0
AOI[likes 4]Hit0
AOI[arrowleft 4]Hit0
AOI[arrowright 4]Hit0
AOI[selection 4]Hit1
AOI[comments 4]Hit0
AOI[pagecount 4]Hit0
AOI[rating 4]Hit0
AOI[text 4]Hit42
AOI[title 4]Hit11
AOI[idea 4]Hit56
AOI[reviews]Hit0
AOI[reviews 2]Hit0
AOI[reviews 3]Hit0
AOI[reviews 4]Hit0

Screen:  2
AOI[selection 5]Hit1
AOI[title 5]Hit31
AOI[text 5]Hit127
AOI[arrowleft 5]Hit3
AOI[pagecount 5]Hit4
AOI[rating 5]Hit2
AOI[likes 5]Hit0
AOI[comments 5]Hit0
AOI[idea 5]Hit176
AOI[arrowright 5]Hit2
AOI[date 5]Hit0
AOI[date 6]Hit0
AOI[likes 6]Hit0
AOI[arrowleft 6]Hit0
AOI[arrowright 6]Hit1
AOI[selection 6]Hit0
AOI[comments 6]Hit0
AOI[pagecount 6]Hit1
AOI[rating 6]Hit1
AOI[text 6]Hit61
AOI[title 6]Hit20
AOI[idea 6]Hit94
AOI[date 7]Hit0
AOI[likes 7]Hit1
AOI[arrowleft 7]Hit0
AOI[arrowright 7]Hit0
AOI

AOI[reviews 30]Hit0
AOI[reviews 31]Hit0
AOI[reviews 32]Hit0
AOI[fullscreen 8]Hit154

Participant:  12
Screen:  1
AOI[Screentitle]Hit0
AOI[selection]Hit0
AOI[title]Hit4
AOI[text]Hit266
AOI[arrowleft]Hit0
AOI[pagecount]Hit9
AOI[rating]Hit2
AOI[likes]Hit0
AOI[comments]Hit0
AOI[idea]Hit300
AOI[fullscreen]Hit499
AOI[arrowright]Hit1
AOI[date]Hit0
AOI[date 2]Hit0
AOI[likes 2]Hit2
AOI[arrowleft 2]Hit0
AOI[arrowright 2]Hit0
AOI[selection 2]Hit0
AOI[comments 2]Hit0
AOI[pagecount 2]Hit0
AOI[rating 2]Hit0
AOI[text 2]Hit34
AOI[title 2]Hit2
AOI[idea 2]Hit41
AOI[date 3]Hit0
AOI[likes 3]Hit0
AOI[arrowleft 3]Hit0
AOI[arrowright 3]Hit0
AOI[selection 3]Hit0
AOI[comments 3]Hit0
AOI[pagecount 3]Hit1
AOI[rating 3]Hit4
AOI[text 3]Hit51
AOI[title 3]Hit13
AOI[idea 3]Hit72
AOI[date 4]Hit0
AOI[likes 4]Hit0
AOI[arrowleft 4]Hit4
AOI[arrowright 4]Hit0
AOI[selection 4]Hit0
AOI[comments 4]Hit0
AOI[pagecount 4]Hit1
AOI[rating 4]Hit1
AOI[text 4]Hit74
AOI[title 4]Hit4
AOI[idea 4]Hit85
AOI[reviews]Hit0
AOI[reviews 2]Hit0

AOI[reviews 22]Hit21
AOI[reviews 23]Hit0
AOI[reviews 24]Hit0
AOI[selection 21]Hit0
AOI[title 21]Hit0
AOI[text 21]Hit84
AOI[arrowleft 21]Hit0
AOI[pagecount 21]Hit343
AOI[rating 21]Hit189
AOI[likes 21]Hit49
AOI[comments 21]Hit99
AOI[idea 21]Hit1345
AOI[arrowright 21]Hit0
AOI[date 21]Hit0
AOI[date 22]Hit0
AOI[likes 22]Hit91
AOI[arrowleft 22]Hit0
AOI[arrowright 22]Hit0
AOI[selection 22]Hit0
AOI[comments 22]Hit105
AOI[pagecount 22]Hit385
AOI[rating 22]Hit315
AOI[text 22]Hit399
AOI[title 22]Hit0
AOI[idea 22]Hit1988
AOI[date 23]Hit0
AOI[likes 23]Hit21
AOI[arrowleft 23]Hit0
AOI[arrowright 23]Hit0
AOI[selection 23]Hit0
AOI[comments 23]Hit0
AOI[pagecount 23]Hit126
AOI[rating 23]Hit0
AOI[text 23]Hit749
AOI[title 23]Hit7
AOI[idea 23]Hit1218
AOI[date 24]Hit0
AOI[likes 24]Hit21
AOI[arrowleft 24]Hit0
AOI[arrowright 24]Hit0
AOI[selection 24]Hit0
AOI[comments 24]Hit0
AOI[pagecount 24]Hit315
AOI[rating 24]Hit0
AOI[text 24]Hit945
AOI[title 24]Hit21
AOI[idea 24]Hit1771
AOI[reviews 21]Hit21
AOI[reviews 22]